In [56]:
import pandas as pd
import numpy as np
import nltk
import textwrap
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
#Para medir la similitud de las oraciones
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
df = pd.read_csv('df_total.csv')

In [58]:
doc = df['news'].sample()

In [59]:
print(doc.iloc[0])

Mucho ha cambiado para el mercado de la televisión en los dos últimos años. A fines de 2020 la alianza Mega  Discovery atravesaba por su mejor momento.Por esos días Guillermo Delfino gerente general de Discovery Networks en una entrevista en Diario Financiero celebraba el rating obtenido por la estación que se mantuvo como líder por casi seis años y adelantaba que saldrían a producir en conjunto en 2021 para reforzar las sinergias. Uno de los focos agregaba sería compartir contenidos con la señal de Discovery que llegaría a Chile al año siguiente.Pero poco de eso ocurrió. La alianza siguió dando frutos en lo comercial se continuó con el modelo de venta de publicidad para Mega y los canales de cable de Discovery. Pero de producciones en conjunto y de la llegada de la señal de streaming nunca más se supo.El tiempo pasó y comenzaron los movimientos en la industria. El 8 de abril pasado se cerró la fusión de dos gigantes y se creó Warner Bros Discovery un nuevo imperio de la TV pero esta v

In [60]:
doc2 = textwrap.fill(doc.iloc[0], replace_whitespace=False, fix_sentence_endings=True)

In [61]:
lineas = doc2.split('.')

In [62]:
lineas = [item for item in lineas if item.strip()]

In [63]:
len(lineas)

46

In [64]:
#Vectorizamos con tfidf
tokenizar = TfidfVectorizer(
    stop_words=stopwords.words('spanish'),
    norm = 'l1', #Con esto estamos diciendo que el valor máximo de cada una de las líneas no puede sumar más de uno (van de 0 a 1)
)

In [65]:
#Construimos la matriz con las lineas
x = tokenizar.fit_transform(lineas)

In [66]:
x

<46x367 sparse matrix of type '<class 'numpy.float64'>'
	with 535 stored elements in Compressed Sparse Row format>

In [67]:
#Matriz de similitud
similitud = cosine_similarity(x)

In [68]:
similitud.shape

(46, 46)

In [69]:
#Normalizar
similitud = similitud/similitud.sum(axis=1, keepdims=True)

In [70]:
similitud

array([[0.61820755, 0.        , 0.03100309, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.45128014, 0.01079313, ..., 0.        , 0.        ,
        0.        ],
       [0.02776436, 0.01324092, 0.55362676, ..., 0.        , 0.        ,
        0.02294118],
       ...,
       [0.        , 0.        , 0.        , ..., 0.95033354, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.40713276,
        0.        ],
       [0.        , 0.        , 0.02229842, ..., 0.        , 0.        ,
        0.53811535]])

In [71]:
similitud[0].sum()

1.0

In [72]:
#Suavizado y regularización
#Matriz de transición uniforme
u = np.ones_like(similitud)/len(similitud)

In [73]:
u

array([[0.02173913, 0.02173913, 0.02173913, ..., 0.02173913, 0.02173913,
        0.02173913],
       [0.02173913, 0.02173913, 0.02173913, ..., 0.02173913, 0.02173913,
        0.02173913],
       [0.02173913, 0.02173913, 0.02173913, ..., 0.02173913, 0.02173913,
        0.02173913],
       ...,
       [0.02173913, 0.02173913, 0.02173913, ..., 0.02173913, 0.02173913,
        0.02173913],
       [0.02173913, 0.02173913, 0.02173913, ..., 0.02173913, 0.02173913,
        0.02173913],
       [0.02173913, 0.02173913, 0.02173913, ..., 0.02173913, 0.02173913,
        0.02173913]])

In [74]:
#matriz de similitud suavizada
factor = 0.15
similitud_suavizada = (1 - factor) * similitud + factor * u

In [75]:
# Verifica si hay NaNs o Infs en la matriz suavizada
print(np.isnan(similitud_suavizada).any())  # Esto debería devolver False
print(np.isinf(similitud_suavizada).any())  # Esto debería devolver False

False
False


In [76]:
#matriz estacionaria 
eigenvals, eigenvecs = np.linalg.eig(similitud_suavizada.T)

In [77]:
eigenvals

array([1.        , 0.8153397 , 0.85      , 0.85      , 0.16921579,
       0.73532234, 0.72148101, 0.70594586, 0.68754862, 0.68004725,
       0.66945273, 0.65454144, 0.6431729 , 0.62853714, 0.61929836,
       0.60776598, 0.59825866, 0.19594073, 0.55380038, 0.54688728,
       0.5306404 , 0.53905257, 0.50957113, 0.21905116, 0.23095466,
       0.23981072, 0.24815559, 0.25322798, 0.46746667, 0.45805734,
       0.44878054, 0.4365941 , 0.42346266, 0.27835949, 0.29070488,
       0.29322531, 0.40640076, 0.38840452, 0.31680571, 0.37572683,
       0.32429728, 0.33179103, 0.34577958, 0.35284293, 0.36332654,
       0.36240862])

In [78]:
eigenvecs[:,0]

array([0.13025198, 0.16243131, 0.13832395, 0.138763  , 0.14584417,
       0.16574867, 0.12272722, 0.12532491, 0.19473604, 0.12308305,
       0.15839434, 0.14029028, 0.13679465, 0.17850507, 0.16033705,
       0.17519889, 0.13340655, 0.15932836, 0.13873505, 0.14584417,
       0.11886363, 0.20316807, 0.17085204, 0.13501553, 0.16573305,
       0.15428668, 0.14674564, 0.1458129 , 0.18000885, 0.12397873,
       0.16620675, 0.15116554, 0.13410374, 0.11653863, 0.12147489,
       0.12070624, 0.12787865, 0.11648951, 0.17243135, 0.11859474,
       0.15173423, 0.11729314, 0.13297605, 0.13564605, 0.16883771,
       0.13822089])

In [79]:
score = eigenvecs[:,0]

In [80]:
sort_idx = np.argsort(-score)

In [81]:
print("Crear resumen:")
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (score[i], lineas[i])))

Crear resumen:
0.20: Sin embargo según fuentes
ligadas a la operación la situación no
es nada de fácil Bethia no
tiene hoy los cerca de US 60 millones que
debe pagarle al nuevo
conglomerado considerando el valor al cual entró
Discovery ajustado
por UF y tipo de cambio
0.19:   El 8 de
abril pasado se cerró la fusión de dos gigantes y se
creó Warner Bros
Discovery un nuevo imperio de la TV pero esta vez
enfocado en el
streaming hoy por hoy el gran rey en el planeta de los
contenidos
0.18:   Por lo mismo las razones
de Discovery para apalancar su
negocio de cable a través de Mega están
súper debilitadas y es mucho
menos útil optar por un canal de TV en un
país cuando ese negocio está
débil y estás migrando a otro foco indica
un conocedor de la industria
0.18: 
Pero existía otra cláusula que se mantuvo en secreto hasta
ahora a los
tres años Discovery podía activar un put una opción de
venta según la
cual Bethia estaba obligada a recomprar su
participación
0.18:   La opción vencía a fines d